### eCRF별 추출된 Raw Data를 eCRF 별로 시트 분리하기

In [1]:
import pandas as pd
import re

# 엑셀 파일 경로
file_path = 'Raw_data_eCRF별.xlsx'

# 데이터 로드
df = pd.read_excel(file_path)

c:\my-project\myenv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df.head()

,STUDY_OID,STUDY_NAME,LOCATION_CODE,LOCATION_NAME,SUBJ_ID,SUBJ_INITIAL,ICF_DT,ARMCRITERIA,ARMID,RANDOMNO,...,LRCTRC,LRCTRCLFO,LROET,LROETN,LROETDY,LROETR,LROETRIFO,LROETRC,LROETRCIFO,LRCMT
0,C23145,조기 위암에서 형광을 이용한 감시림프절 위 보존 수술의 유용성 평가,CMCEP,가톨릭대학교 은평성모병원,CMCEP-002,LBS,2024-01-08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C23145,조기 위암에서 형광을 이용한 감시림프절 위 보존 수술의 유용성 평가,CMCEP,가톨릭대학교 은평성모병원,CMCEP-002,LBS,2024-01-08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C23145,조기 위암에서 형광을 이용한 감시림프절 위 보존 수술의 유용성 평가,CMCEP,가톨릭대학교 은평성모병원,CMCEP-002,LBS,2024-01-08,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C23145,조기 위암에서 형광을 이용한 감시림프절 위 보존 수술의 유용성 평가,CMCEP,가톨릭대학교 은평성모병원,CMCEP-002,LBS,2024-01-08,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C23145,조기 위암에서 형광을 이용한 감시림프절 위 보존 수술의 유용성 평가,CMCEP,가톨릭대학교 은평성모병원,CMCEP-002,LBS,2024-01-08,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 공통 열 (0 ~ 20열)
common_cols = df.columns[:21]

# 이후 열들
all_cols = df.columns
extra_cols = all_cols[21:]

# 'CRF_VER_NM_'으로 시작하는 열 찾기
split_points = [i for i, col in enumerate(all_cols) if str(col).startswith('CRF_VER_NM_')]

# eCRF 구간 나누기
sheet_ranges = []
for i in range(len(split_points)):
    start = split_points[i] # 시작점:'CRF_VER_NM_'으로 시작하는 열의 인덱스
    end = split_points[i + 1] if i + 1 < len(split_points) else len(all_cols) # 끝점: 다음번 'CRF_VER_NM_' 열의 인덱스 전까지. 또는 마지막 열
    sheet_ranges.append((start, end))

In [13]:
# 저장
with pd.ExcelWriter('C:/my-project/my-project/my-project/DM_JOB/Raw_data_cleaning/C23145_Separated_Raw_data.xlsx', engine='openpyxl') as writer:
    for idx, (start, end) in enumerate(sheet_ranges):
        # 공통 열 + 해당 eCRF 열만 선택
        selected_cols = list(common_cols) + list(all_cols[start:end])
        df_sheet = df[selected_cols]

        # CRF_NM_로 시작하는 열 찾기
        crf_nm_col = [col for col in df_sheet.columns if str(col).startswith('CRF_NM_')]

        if crf_nm_col:
            crf_nm_values = df_sheet[crf_nm_col[0]].unique()  # CRF_NM 열에서 고유 값 가져오기

            if len(crf_nm_values) == 1:  # 값이 하나일 경우 그 값으로 시트명 지정
                sheet_name = crf_nm_values[0]
            else:
                # 하나의 시트에 CRF_NM 값이 여러 개일 경우
                print(f"Warning: CRF_NM 값이 여러 개임. 시트 {idx+1}에서 CRF_NM 값들: {crf_nm_values}")
                # 예시: 'CRF_NM_1_1' 같은 방식으로 시트명 지정
                sheet_name = f"CRF_NM_{idx+1}_Multiple"

            # 시트명 최대 31자임
            sheet_name = sheet_name[:31]
        else:
            # 하나의 시트에 CRF_NM 열이 없는 경우
            print(f"Error: CRF_NM 열이 없음. 시트 {idx+1}에는 CRF_NM_* 열이 없습니다.")
            sheet_name = f"Error_CRFMissing_{idx+1}"

        # CRF_STAT_로 시작하는 열이 'C'인 행만 남김
        stat_cols = [col for col in df_sheet.columns if str(col).startswith('CRF_STAT_')]
        if stat_cols:  # 해당 열이 존재할 경우에만 필터링
            df_sheet = df_sheet[df_sheet[stat_cols].eq('C').any(axis=1)]

        # 시트 저장
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

* EE CRF는 미작성 대상자 포함하여 별도 파일로 제공!